# Библиотеки

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as sps

# Загрузка и обработка данных

In [20]:
df_btc_20 = pd.read_excel(r"C:\Users\Владимир\Desktop\bitcoin_2020.xlsx").sort_values('Дата').reset_index(drop=True)
df_btc_20

,Дата,Значение
0,2020-01-01,7200.17000
1,2020-01-02,6985.47000
2,2020-01-03,7344.88000
3,2020-01-04,7410.66000
4,2020-01-05,7411.32000
...,...,...
361,2020-12-27,26366.62790
362,2020-12-28,26740.00980
363,2020-12-29,26898.76162
364,2020-12-30,28914.69584


In [11]:
#расчет исторической доходности

#конвертация в массив NP
p = np.array(df_btc_20.iloc[:,1:2])

#массив доходности
r = p[1:]/p[:-1] - 1

#средняя геометрическая дневная доходность
mu = np.prod(1+r)**(1/len(r)) - 1

#стандартное отклонения
sigma = np.std(r)

## Расчёт рисков (VAR)

In [12]:
#вкладываемая сумма в валюте
value = 1000000

#сколько процентов наихудших исходов мы расмматриваем для расчет показателей (в процентном выражении)
var = 0.99

#доверительный интервал
a = 1 - var

In [19]:
#параметрический VAR
VCV_var = -value * (mu + sigma*sps.norm.ppf(a))
print('Параметрические Value at Risk ' + str(round(VCV_var, 2)))

#исторический VAR
HS_var = -value*np.quantile(r, a)
print('Исторический Value at Risk ' + str(round(HS_var, 2)))

#симуляция Монте-Карло

#количество симуляций
n = 1000

#задаем дрифт
drift = mu - sigma**2/2

#массив симуляций
sim = np.random.normal(drift, sigma, n)

#один процент наихудших исходов
MCS_var = -value*np.quantile(sim, a)
print('Монте-Карло Value at Risk ' + str(round(MCS_var, 2)))


Параметрические Value at Risk 83096.14
Исторический Value at Risk 88264.19
Монте-Карло Value at Risk 81980.52


# Функция

In [29]:
#для одного датасета
def var(df):
    p = np.array(df.iloc[:,1:2])
    r = p[1:]/p[:-1] - 1
    mu = np.prod(1+r)**(1/len(r)) - 1
    sigma = np.std(r)
    value = 1000000
    var = 0.99
    a = 1 - var
    VCV_var = -value * (mu + sigma*sps.norm.ppf(a))
    print('Параметрические Value at Risk ' + str(round(VCV_var, 2)))
    HS_var = -value*np.quantile(r, a)
    print('Исторический Value at Risk ' + str(round(HS_var, 2)))
    n = 1000
    drift = mu - sigma**2/2
    sim = np.random.normal(drift, sigma, n)
    MCS_var = -value*np.quantile(sim, a)
    print('Монте-Карло Value at Risk ' + str(round(MCS_var, 2)))
    
#для нескольких датасетов
def var_df_list(df_list):
    for df in df_list:
        p = np.array(df.iloc[:,1:2])
        r = p[1:]/p[:-1] - 1
        mu = np.prod(1+r)**(1/len(r)) - 1
        sigma = np.std(r)
        value = 1000000
        var = 0.99
        a = 1 - var
        VCV_var = -value * (mu + sigma*sps.norm.ppf(a))
        print('Параметрические Value at Risk ' + str(round(VCV_var, 2)))
        HS_var = -value*np.quantile(r, a)
        print('Исторический Value at Risk ' + str(round(HS_var, 2)))
        n = 1000
        drift = mu - sigma**2/2
        sim = np.random.normal(drift, sigma, n)
        MCS_var = -value*np.quantile(sim, a)
        print('Монте-Карло Value at Risk ' + str(round(MCS_var, 2)))
        print()

In [24]:
#проверка
var(df_btc_20)

Параметрические Value at Risk 83096.14
Исторический Value at Risk 88264.19
Монте-Карло Value at Risk 84267.44


# Расчет на примере Газпрома

In [27]:
#проверка на газпроме
df_gazp_20 = pd.read_csv(r"C:\Users\Владимир\Desktop\GAZP_2020.csv", sep=';')
df_gazp_20['<DATE>'] = pd.to_datetime(df_gazp_20['<DATE>'], format='%Y%m%d')
df_gazp_20 = df_gazp_20[['<DATE>', '<CLOSE>']]

df_gazp_21 = pd.read_csv(r"C:\Users\Владимир\Desktop\GAZP_2021.csv", sep=';')
df_gazp_21['<DATE>'] = pd.to_datetime(df_gazp_21['<DATE>'], format='%Y%m%d')
df_gazp_21 = df_gazp_21[['<DATE>', '<CLOSE>']]

df_gazp_22 = pd.read_csv(r"C:\Users\Владимир\Desktop\GAZP_2022.csv", sep=';')
df_gazp_22['<DATE>'] = pd.to_datetime(df_gazp_22['<DATE>'], format='%Y%m%d')
df_gazp_22 = df_gazp_22[['<DATE>', '<CLOSE>']]

GAZP_list = [df_gazp_20, df_gazp_21, df_gazp_22]

In [28]:
df_gazp_22

,<DATE>,<CLOSE>
0,2022-01-03,353.73
1,2022-01-04,352.55
2,2022-01-05,335.00
3,2022-01-06,346.13
4,2022-01-10,344.00
...,...,...
163,2022-09-22,231.45
164,2022-09-23,224.84
165,2022-09-26,206.00
166,2022-09-27,217.30


In [30]:
var_df_list(GAZP_list)

Параметрические Value at Risk 44536.45
Исторический Value at Risk 54046.56
Монте-Карло Value at Risk 48742.94

Параметрические Value at Risk 36936.69
Исторический Value at Risk 36857.17
Монте-Карло Value at Risk 34538.14

Параметрические Value at Risk 125171.19
Исторический Value at Risk 199436.6
Монте-Карло Value at Risk 123461.92

